# Neural Network Hyperparameter Grid Search

In [1]:
import tensorflow as tf
import random
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import warnings
from sklearn.metrics import r2_score
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.utils import shuffle
from tqdm import tqdm

2023-02-26 00:09:28.260137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 00:09:28.360289: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-26 00:09:28.898787: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/tychi/miniconda3/lib/
2023-02-26 00:09:28.898874: W tensorflow/compiler/xla/stre

In [2]:
training_data = pd.read_csv('../Datasets/training_data_full.csv')
training_data = shuffle(training_data)

In [3]:
non_numerical_features = set([
    'StateAbbr','StateDesc','CityName','CountyFIPS','TractFIPS'
])

target_features = set(['DIABETES_3Y_Change_Percentage'])

numerical_features = list(
    set(training_data.columns) - non_numerical_features - target_features
)

features = training_data[numerical_features]
labels = training_data['DIABETES_3Y_Change_Percentage']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)

In [5]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(features) # technically should just do this on X_train

2023-02-26 00:09:30.554065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-26 00:09:30.558506: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-26 00:09:30.558716: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-26 00:09:30.559252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them i

In [6]:
def build_model(layer_config, learning_rate=0.001, add_dropout=False):
    model = tf.keras.Sequential([
        normalizer
    ])
    
    for layer_size in layer_config:
        # dense layer
        model.add(tf.keras.layers.Dense(layer_size, activation='relu'))
        
        # some studies suggest adding dropout layer after each dense layer
        if add_dropout: model.add(tf.keras.layers.Dropout(0.2))
    
    # activation layer
    model.add(tf.keras.layers.Dense(1, activation='linear'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mean_squared_error"
    )
    return model

# Grid Search

In [7]:
layer_config_options = [
    [50,25,5],
    [30,15,3],
    [20,10,2],
    [20,5]
]

learning_rate_options = [
    0.1,
    0.01,
    0.001,
]

add_dropout_options = [
    True,
    False
]

batch_size_options = [
    2, 16, 32
]

early_stop_callback = EarlyStopping(monitor='val_loss', patience=3)

In [8]:
combination_count =\
    len(layer_config_options) * len(learning_rate_options) *\
    len(add_dropout_options) * len(batch_size_options)

print(f'Combinations to try: {combination_count}')

tqdm._instances.clear() 
pbar = tqdm(total=combination_count) 

result_list = [] # later to turn into dataframe

for layer_config in layer_config_options:
    for learning_rate in learning_rate_options:
        for add_dropout in add_dropout_options:
            for batch_size in batch_size_options:
                nn_model = build_model(
                    layer_config, learning_rate, add_dropout
                )
                
                nn_model.fit(
                    X_train,
                    y_train,
                    validation_data=(X_test, y_test),
                    batch_size=batch_size,
                    epochs=50, # there is early stop, epoch not an issue
                    callbacks=[early_stop_callback],
                    verbose=0
                )
                
                # make predictions
                y_test_predictions = nn_model.predict(X_test)
                y_train_predictions = nn_model.predict(X_train)
                
                # calculate r2 scores
                train_r2 = r2_score(y_train, y_train_predictions)
                test_r2 = r2_score(y_test, y_test_predictions)
                
                result_list.append({
                    'layer_config': str(layer_config),
                    'learning_rate': learning_rate,
                    'add_dropout': add_dropout,
                    'batch_size': batch_size,
                    'train_r2': train_r2,
                    'test_r2': test_r2
                })
                
                pbar.update(1)

Combinations to try: 72


  0%|                                                                                            | 0/72 [00:00<?, ?it/s]2023-02-26 00:09:35.222398: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-26 00:09:35.229876: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1fc9d700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-26 00:09:35.229906: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-02-26 00:09:35.234696: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-26 00:09:35.329002: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-26 00:09:35.357372:

568/568 [==============================] - 2s 3ms/step


  1%|█                                                                               | 1/72 [14:28<17:07:16, 868.12s/it]

568/568 [==============================] - 2s 3ms/step


...

 97%|█████████████████████████████████████████████████████████████████████████████▊  | 70/72 [6:11:18<18:49, 564.96s/it]

568/568 [==============================] - 1s 2ms/step


 99%|██████████████████████████████████████████████████████████████████████████████▉ | 71/72 [6:14:03<07:25, 445.15s/it]

568/568 [==============================] - 1s 2ms/step


100%|████████████████████████████████████████████████████████████████████████████████| 72/72 [6:15:36<00:00, 339.36s/it]

In [ ]:
result_df = pd.DataFrame(result_list)

In [10]:
result_df.to_csv('./NeuralNetworkResults.csv', index=False)

In [28]:
result_df = pd.read_csv('./NeuralNetworkResults.csv')

In [29]:
result_df = result_df.sort_values(by=['test_r2'], ascending=False).reset_index(drop=True)

In [31]:
result_df = result_df.round(4)

In [35]:
result_df.to_csv('NeuralNetwork_GridSearch_Results_Sorted.csv', index=False)